In [313]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [314]:
symbol = "BTC-USDT"
folder_path = f'./data'
file_path = f'{folder_path}/{symbol}.parquet'

df = pd.read_parquet(file_path, engine='pyarrow')

df = df.reset_index(drop=True)

print(df.info())

df['open'] = df['open'].replace(0, 0.00000001)
df['high'] = df['high'].replace(0, 0.00000001)
df['low'] = df['low'].replace(0, 0.00000001)
df['close'] = df['close'].replace(0, 0.00000001)
df['volume'] = df['volume'].replace(0, 0.00000001)
df['trades'] = df['trades'].replace(0, 1)

print(df.describe(include='all'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123248 entries, 0 to 123247
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   time    123248 non-null  datetime64[ns]
 1   open    123248 non-null  float64       
 2   high    123248 non-null  float64       
 3   low     123248 non-null  float64       
 4   close   123248 non-null  float64       
 5   volume  123248 non-null  float64       
 6   trades  123248 non-null  int64         
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 6.6 MB
None
                                time           open           high  \
count                         123248  123248.000000  123248.000000   
mean   2023-08-03 00:22:04.334674432   31461.676669   31485.989206   
min              2023-01-01 00:00:00   16508.940000   16512.000000   
25%              2023-04-18 00:58:45   26219.035000   26239.270000   
50%              2023-08-03 00:37:30   28571.070000   28594.5

In [315]:
LB = 384
LF = 288

In [316]:
exchange = 1
symbolId = 680
timeframe = 5
model = 1

df["modelId"] = model
df["exchangeId"] = exchange
df["symbolId"] = symbolId

In [317]:
# df["time"] = pd.to_datetime(df["time"], unit="s") + pd.Timedelta(hours=3)
df["time"] = pd.to_datetime(df["time"], unit="s")

# df = df.sort_values(by='time').reset_index(drop=True).tail(10000)
# df = df.tail(10000)

# Вычисление и добавление новых столбцов
df["tday_year"] = df["time"].dt.dayofyear / 366.0
df["tday_month"] = (df["time"].dt.day) / 31.0
df["tday_week"] = (df["time"].dt.dayofweek + 1) / 7
df["tmonth_year"] = df["time"].dt.month / 12.0
df["tweek_year"] = df["time"].dt.isocalendar().week / 53.0
df["tsecond_day"] = (df["time"].dt.hour * 3600 + df["time"].dt.minute * 60 + df["time"].dt.second) / 86400.0
# df.reindex()

In [318]:
# Вычисление скользящего среднего
df["price_ma"] = df["open"].rolling(window=89, min_periods=1).mean()
df['price_ma'] = df['price_ma'].replace(0, 0.0000000000000001)

# Вычисление скользящего среднего для "volume"
df["volume_ma"] = df["volume"].rolling(window=89, min_periods=1).mean()
df['volume_ma'] = df['volume_ma'].replace(0, 0.0000000000000001)

df["topen"] = (df["open"] / df["price_ma"] - 1)
df["thigh"] = (df["high"] / df["price_ma"] - 1)
df["tlow"] = (df["low"] / df["price_ma"] - 1)
df["tclose"] = (df["close"] / df["price_ma"] - 1)
df['tvolume'] = (df['volume'] / df['volume_ma'] - 1)

df['lpopen'] = df['open'].pct_change(periods=1)
df['lfopen'] = df['open'].pct_change(periods=LB)

df['lphigh'] = df['high'].pct_change(periods=1)
df['lfhigh'] = df['high'].pct_change(periods=LB)

df['lplow'] = df['low'].pct_change(periods=1)
df['lflow'] = df['low'].pct_change(periods=LB)

df['lpopen'] = df['open'].pct_change(periods=1)
df['lfopen'] = df['open'].pct_change(periods=LB)

df['lpclose'] = df['close'].pct_change(periods=1)
df['lfclose'] = df['close'].pct_change(periods=LB)

df['lpvolume'] = df['volume'].pct_change(periods=1)
df['lfvolume'] = df['volume'].pct_change(periods=LB)

df['lppricema'] = df['price_ma'].pct_change(periods=1)
df['lfpricema'] = df['price_ma'].pct_change(periods=LB)

df['lpvolumema'] = df['volume_ma'].pct_change(periods=1)
df['lfvolumema'] = df['volume_ma'].pct_change(periods=LB)

In [319]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123248 entries, 0 to 123247
Data columns (total 37 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   time         123248 non-null  datetime64[ns]
 1   open         123248 non-null  float64       
 2   high         123248 non-null  float64       
 3   low          123248 non-null  float64       
 4   close        123248 non-null  float64       
 5   volume       123248 non-null  float64       
 6   trades       123248 non-null  int64         
 7   modelId      123248 non-null  int64         
 8   exchangeId   123248 non-null  int64         
 9   symbolId     123248 non-null  int64         
 10  tday_year    123248 non-null  float64       
 11  tday_month   123248 non-null  float64       
 12  tday_week    123248 non-null  float64       
 13  tmonth_year  123248 non-null  float64       
 14  tweek_year   123248 non-null  Float64       
 15  tsecond_day  123248 non-null  floa

In [320]:
# Вычисляем Profit и DD для BUY в будущем
df = df.reset_index(drop=True)

print("DF Length",len(df))

for i, row in df.iterrows():
    close = row['close']

    if i + LF + 1 < len(df):
        # Рассчитываем значение целей для BUY
        max_value = df['high'][i+1:i+LF].max()
        max_index = df['high'][i+1:i+LF].argmax()

        min_value = df['low'][i:max([i+1, max_index])].min()
        min_index = df['low'][i:max([i+1, max_index])].argmin()

        df.loc[i, 'buy_profit'] = max_value / close - 1
        df.loc[i, 'buy_dd'] =  min_value / close - 1
        df.loc[i, 'buy_time'] = max_index / LF
        df.loc[i, 'buy_dd_time'] = min_index / LF

        # Рассчитываем значение целей для SELL
        max_value = df['low'][i+1:i+LF].min()
        max_index = df['low'][i+1:i+LF].argmin()

        min_value = df['high'][i:max([i+1, max_index])].max()
        min_index = df['high'][i:max([i+1, max_index])].argmax()

        df.loc[i, 'sell_profit'] = close / max_value - 1
        df.loc[i, 'sell_dd'] = close / min_value - 1
        df.loc[i, 'sell_time'] = max_index / LF
        df.loc[i, 'sell_dd_time'] = min_index / LF
    else:
        df.loc[i, 'buy_profit'] = np.nan
        df.loc[i, 'buy_dd'] = np.nan
        df.loc[i, 'buy_time'] = np.nan
        df.loc[i, 'buy_dd_time'] = np.nan
        df.loc[i, 'sell_profit'] = np.nan
        df.loc[i, 'sell_dd'] = np.nan
        df.loc[i, 'sell_time'] = np.nan
        df.loc[i, 'sell_dd_time'] = np.nan

print(df[['buy_profit', 'buy_dd', 'buy_time', 'buy_dd_time']].head(LF + 3))
print(df[['sell_profit', 'sell_dd', 'sell_time', 'sell_dd_time']].head(LF + 3))
print(df[['buy_profit', 'buy_dd', 'buy_time', 'buy_dd_time']].tail(LF + 3))
print(df[['sell_profit', 'sell_dd', 'sell_time', 'sell_dd_time']].tail(LF + 3))


DF Length 123248
     buy_profit    buy_dd  buy_time  buy_dd_time
0      0.005601 -0.002200  0.847222     0.302083
1      0.006131 -0.001674  0.843750     0.298611
2      0.006559 -0.001312  0.993056     0.295139
3      0.005692 -0.002173  0.989583     0.291667
4      0.005655 -0.002209  0.986111     0.288194
..          ...       ...       ...          ...
286    0.011405 -0.000333  0.916667     0.000000
287    0.010982 -0.000455  0.913194     0.000000
288    0.010770 -0.000209  0.909722     0.000000
289    0.010898 -0.000156  0.906250     0.000000
290    0.011233 -0.000156  0.902778     0.000000

[291 rows x 4 columns]
     sell_profit   sell_dd  sell_time  sell_dd_time
0       0.002204 -0.001473   0.298611      0.107639
1       0.001676 -0.001999   0.295139      0.104167
2       0.001314 -0.002360   0.291667      0.100694
3       0.002178 -0.001499   0.288194      0.097222
4       0.002214 -0.001463   0.284722      0.093750
..           ...       ...        ...           ...
286    

In [321]:
# Вычисляем Profit и DD для BUY print("Len", в прошлом LF днях
df = df.reset_index(drop=True)

print("shifted", len(df))

for i, row in df.iterrows():
    close = row['close']
    # print(i, close)

    if i >= LF:
        # Рассчитываем значение целей для BUY
        max_value = df['high'][i-LF+1:i].max()
        max_index = df['high'][i-LF+1:i].argmax()

        min_value = df['low'][i-LF:i].min()
        min_index = df['low'][i-LF:i].argmin()

        df.loc[i, 'past_buy_profit'] = max_value / close - 1
        df.loc[i, 'past_buy_dd'] =  min_value / close - 1
        df.loc[i, 'past_buy_time'] = max_index / LF
        df.loc[i, 'past_buy_dd_time'] = min_index / LF

        # Рассчитываем значение целей для SELL
        max_value = df['low'][i-LF+1:i].min()
        max_index = df['low'][i-LF+1:i].argmin()

        min_value = df['high'][i-LF:i].max()
        min_index = df['high'][i-LF:i].argmax()

        df.loc[i, 'past_sell_profit'] = close / max_value - 1
        df.loc[i, 'past_sell_dd'] = close / min_value - 1
        df.loc[i, 'past_sell_time'] = max_index / LF
        df.loc[i, 'past_sell_dd_time'] = min_index / LF
    else:
        df.loc[i, 'past_buy_profit'] = np.nan
        df.loc[i, 'past_buy_dd'] = np.nan
        df.loc[i, 'past_buy_time'] = np.nan
        df.loc[i, 'past_buy_dd_time'] = np.nan
        df.loc[i, 'past_sell_profit'] = np.nan
        df.loc[i, 'past_sell_dd'] = np.nan
        df.loc[i, 'past_sell_time'] = np.nan
        df.loc[i, 'past_sell_dd_time'] = np.nan

    # if (i > len(df) - LF - 3) or (i < LF + 3):
        # print(i, close, max_value, max_index, min_value, min_index)

print(df[['past_buy_profit', 'past_buy_dd', 'past_buy_time', 'past_buy_dd_time']].head(LF+3))
print(df[['past_sell_profit', 'past_sell_dd', 'past_sell_time', 'past_sell_dd_time']].head(LF+3))
print(df[['past_buy_profit', 'past_buy_dd', 'past_buy_time', 'past_buy_dd_time']].tail(LF+3))
print(df[['past_sell_profit', 'past_sell_dd', 'past_sell_time', 'past_sell_dd_time']].tail(LF+3))


shifted 123248
     past_buy_profit  past_buy_dd  past_buy_time  past_buy_dd_time
0                NaN          NaN            NaN               NaN
1                NaN          NaN            NaN               NaN
2                NaN          NaN            NaN               NaN
3                NaN          NaN            NaN               NaN
4                NaN          NaN            NaN               NaN
..               ...          ...            ...               ...
286              NaN          NaN            NaN               NaN
287              NaN          NaN            NaN               NaN
288         0.000468    -0.007294       0.847222          0.302083
289         0.000595    -0.007167       0.843750          0.298611
290         0.000989    -0.006839       0.993056          0.295139

[291 rows x 4 columns]
     past_sell_profit  past_sell_dd  past_sell_time  past_sell_dd_time
0                 NaN           NaN             NaN                NaN
1              

In [322]:
# # Рассчитываем максимальное значение high за окно
# df['max_high_in_window'] = df['high'].rolling(window=LF).max()
# # Рассчитываем колонку buy_profit
# df['buy_profit'] = (df['max_high_in_window'] / df['close']) - 1
# # Применяем ограничение значений в диапазоне от 0 до 1
# df['buy_profit'] = df['buy_profit'].clip(0, 1)
# # Удаляем временную колонку max_high_in_window
# df.drop('max_high_in_window', axis=1, inplace=True)

# # Рассчитываем минимальное значение low за окно
# df['min_low_in_window'] = df['lowx'].rolling(window=LF).min()
# # Рассчитываем колонку buy_drawdown
# df['buy_drawdown'] = (df['close'] / df['min_low_in_window']) - 1
# # Применяем ограничение значений в диапазоне от 0 до 1
# df['buy_drawdown'] = df['buy_drawdown'].clip(0, 1)
# # Удаляем временную колонку min_low_in_window
# df.drop('min_low_in_window', axis=1, inplace=True)

# # sell_profit = buy_drawdown
# df['sell_profit'] = df['buy_drawdown']
# # sell_drawdown = buy_profit
# df['sell_drawdown'] = df['buy_profit']

# # Выполните цикл по строкам DataFrame, начиная с индекса 0
# for i in range(len(df)):
#     # Находите индекс максимального значения в столбце 'high' за последние 288 строк
#     max_index = df['high'].iloc[max(0, i):i + 288].idxmax()  # Увеличиваем диапазон на 1

#     # Записывайте индекс в новую колонку 'idx'
#     df.at[i, 'idx'] = max_index

# df['buy_time'] = (df['idx'] - df.index.values) / 288

# # Выполните цикл по строкам DataFrame, начиная с индекса 0
# for i in range(len(df)):
#     # Находите индекс максимального значения в столбце 'high' за последние 288 строк
#     max_index = df['low'].iloc[max(0, i):i + 288].idxmin()  # Увеличиваем диапазон на 1

#     # Записывайте индекс в новую колонку 'idx'
#     df.at[i, 'idx'] = max_index
# df['sell_time'] = (df['idx'] - df.index.values) / 288

# # cbuy_profit
# df['max_high_buy_profit'] = df['high'].rolling(window=288).max()


# def determine_category(profit):
#     # Определение категорий в соответствии с заданным рядом
#     categories = [1, 2, 3, 5, 8, 13, 21, 34]

#     # Проверка каждой категории
#     for category in categories:
#         if profit <= category:
#             return category
#     return categories[-1]


# df['difference_percent'] = ((df['max_high_buy_profit'] - df['high']) / df['high']) * 100
# df['cbuy_profit'] = df['difference_percent'].apply(determine_category)

# # csell_profit
# df['min_low_in_window'] = df['low'].rolling(window=288).min()


# def determine_category(profit):
#     # Определение категорий в соответствии с заданным рядом
#     categories = [1, 2, 3, 5, 8, 13, 21, 34]

#     # Проверка каждой категории
#     for category in categories:
#         if profit <= category:
#             return category
#     return categories[-1]


# df['difference_percent'] = ((df['low'] - df['min_low_in_window']) / df['min_low_in_window']) * 100
# df['csell_profit'] = df['difference_percent'].apply(determine_category)

# # csell_profit = cbuy_drawdown
# df['csell_drawdown'] = df['cbuy_profit']
# # cbuy_profit = csell_drawdown
# df['cbuy_drawdown'] = df['csell_profit']

# df['cbuy_time'] = (df['buy_time'] * 12).round(0).astype(int)
# df['csell_time'] = (df['sell_time'] * 12).round(0).astype(int)

In [323]:
# Условия для фильтрации строк
buy_condition = (df['buy_profit'] > 0.05) & (df['buy_dd'] < 0.01)
sell_condition = (df['sell_profit'] > 0.05) & (df['sell_dd'] < 0.01)
other_condition_buy = (df['buy_profit'] < 0.01) & (df['buy_dd'] < 0.01)
other_condition_sell = (df['sell_profit'] < 0.01) & (df['sell_dd'] < 0.01)

# Присвоение значений столбцу 'csignal' для строк, удовлетворяющих условиям
df.loc[buy_condition, 'csignal'] = 1
df.loc[sell_condition, 'csignal'] = 3
df.loc[other_condition_buy, 'csignal'] = 0
df.loc[other_condition_sell, 'csignal'] = 4
df['csignal'] = df['csignal'].fillna(2).astype(int)

In [324]:
# добавление timestamp
df['timestamp'] = df['time'].apply(lambda x: int(x.timestamp()))

# pd.to_datetime(df['time'])
# df["time"] = (pd.to_datetime(df["time"]) - pd.Timedelta(hours=3) - pd.Timestamp("1970-01-01")) // pd.Timedelta(seconds=1)
# df.rename(columns={'time': 'timestamp'}, inplace=True)

In [325]:
# обрезаем датасет по краям
df = df.iloc[LB:len(df)-LF-1]

In [326]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Предположим, что df - ваш DataFrame
# scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = MinMaxScaler(feature_range=(-0.99999999999999, 0.99999999999999))

columns_to_scale = ['tvolume', 'lpvolumema', 'lfvolumema', 'lpvolume', 'lfvolume']

if columns_to_scale:
    data_to_scale = df[columns_to_scale]
    scaler.fit(data_to_scale)
    
    # Масштабировать только указанные столбцы и сохранить индексы
    df_scaled = pd.DataFrame(scaler.transform(data_to_scale), columns=columns_to_scale, index=df.index)
    # df_scaled = df_scaled.round(decimals=14) # Округлить до 10 знаков после запятой
    # df_scaled = df_scaled.clip(lower=-1, upper=1)
    df[columns_to_scale] = df_scaled
else:
    scaler.fit(df)
    df_scaled = pd.DataFrame(scaler.transform(df), columns=df.columns, index=df.index)
    # df_scaled = df_scaled.round(decimals=14) # Округлить до 10 знаков после запятой
    # df_scaled = df_scaled.clip(lower=-1, upper=1)
    df = df_scaled

# df.reindex()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122575 entries, 384 to 122958
Data columns (total 55 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   time               122575 non-null  datetime64[ns]
 1   open               122575 non-null  float64       
 2   high               122575 non-null  float64       
 3   low                122575 non-null  float64       
 4   close              122575 non-null  float64       
 5   volume             122575 non-null  float64       
 6   trades             122575 non-null  int64         
 7   modelId            122575 non-null  int64         
 8   exchangeId         122575 non-null  int64         
 9   symbolId           122575 non-null  int64         
 10  tday_year          122575 non-null  float64       
 11  tday_month         122575 non-null  float64       
 12  tday_week          122575 non-null  float64       
 13  tmonth_year        122575 non-null  float6

In [327]:
desired_order = ['modelId', 'exchangeId', 'symbolId',
                 'timestamp', 'open', 'high', 'low', 'close', 'volume',
                 'tday_year', 'tday_month', 'tday_week', 'tmonth_year', 'tweek_year', 'tsecond_day',
                 'price_ma', 'volume_ma',
                 'past_buy_profit', 'past_buy_dd', 'past_buy_time', 'past_buy_dd_time',
                 'past_sell_profit', 'past_sell_dd', 'past_sell_time', 'past_sell_dd_time',
                 'buy_profit', 'buy_dd', 'buy_time', 'buy_dd_time', 'sell_profit', 'sell_dd', 'sell_time', 'sell_dd_time',
                 'csignal',
                                 'topen', 'thigh', 'tlow', 'tclose', 'tvolume',
                 'lpopen', 'lfopen', 'lphigh', 'lfhigh', 'lplow', 'lflow', 'lpvolume', 'lfvolume',
                 'lppricema', 'lfpricema', 'lpvolumema', 'lfvolumema', 'lfclose', 'lpclose']

df = df[desired_order]

In [328]:
folder_path = f'~/code/AI/Pancakeswap'
dataset_file = f'{folder_path}/dataset_{symbol}.parquet'
print(f'Путь к файлу: {dataset_file}')

df.to_parquet(dataset_file, engine='pyarrow', compression='brotli')

Путь к файлу: ~/code/AI/Pancakeswap/dataset_BTC-USDT.parquet


In [329]:
df_min = df.min(axis=0)
df_max = df.max(axis=0)

columns_to_show = []
for col in df.columns:
    try:
        # Проверка, является ли элемент типом Timestamp
        if isinstance(df_min[col], pd.Timestamp) or isinstance(df_max[col], pd.Timestamp):
            # Конвертация Timestamp в Unix timestamp (число) для сравнения, если это подходит для вашей задачи
            min_value = df_min[col].timestamp() if isinstance(df_min[col], pd.Timestamp) else df_min[col]
            max_value = df_max[col].timestamp() if isinstance(df_max[col], pd.Timestamp) else df_max[col]
        else:
            # Если элементы не являются Timestamp, используется исходное значение
            min_value = df_min[col]
            max_value = df_max[col]

        if min_value < -1 or max_value > 1:
            # if col == 'lfvolumema':
            #     print('lfvolumema: ', min_value, max_value)
            columns_to_show.append(col)
    except Exception as e:
        print('Error columt: ', col)
        print(e)
        pass

print(f"Колонки с min < -1 или max > 1: {columns_to_show}")


Колонки с min < -1 или max > 1: ['symbolId', 'timestamp', 'open', 'high', 'low', 'close', 'volume', 'price_ma', 'volume_ma', 'csignal']


In [330]:
# print(df.head())

print(df.tail(10))

print(df.info())

print(df.describe())

        modelId  exchangeId  symbolId   timestamp      open      high  \
122949        1           1       680  1709420700  61821.63  61890.04   
122950        1           1       680  1709421000  61890.03  61917.01   
122951        1           1       680  1709421300  61910.00  62010.00   
122952        1           1       680  1709421600  62009.99  62048.92   
122953        1           1       680  1709421900  62048.92  62050.36   
122954        1           1       680  1709422200  62013.13  62071.76   
122955        1           1       680  1709422500  61982.86  62052.72   
122956        1           1       680  1709422800  62019.21  62063.99   
122957        1           1       680  1709423100  62037.61  62037.62   
122958        1           1       680  1709423400  62025.31  62030.19   

             low     close    volume  tday_year  ...     lplow     lflow  \
122949  61821.63  61890.04  75.98652   0.169399  ...  0.002215 -0.000189   
122950  61863.45  61910.00  53.75758   0.169